PDF Query Using Langchain

In [ ]:
!pip install langchain
!pip install PyPDF2 # READ FROM PDF
!pip install openai
!pip install faiss-cpu #
!pip install tiktoken # Dependency - to create tokens

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community # Install the missing langchain-community package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings # measures the relatedness of text strings
from langchain.text_splitter import CharacterTextSplitter # content inside pdf would be split based on spl characters/text size - done to have fixed size tokens
from langchain.vectorstores import FAISS # vector database - to store

In [ ]:
import os
os.environ["OPENAI_API_KEY"]  = "ADD YOUR API KEY"

In [10]:
 # PROVIDE THE PATH OF PDF FILE/FILES
 pdfreader = PdfReader('bh1.pdf')

In [11]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [12]:
raw_text

"Key Features \nof\nBudget\n2024-2025\nFebruary, 2024ºÉiªÉàÉä´É VÉªÉiÉäViksit Bharat by 2047\nDevelopment Mantra\nComprehensive \ndevelopment of allVision: Prosperous Bharat in harmony with nature, modern \ninfrastructure and opportunities for all\nSabka Saath, \nSabka Vikas\nSabka Saath, \nSabka Vikas, \nSabka Vishwas\nTrinity of demography, \ndemocracy and diversity, \nbacked by ‘Sabka Prayas ’\nDeveloped India \n@ 2047\n1People -Centric Inclusive Development\nSubstantive development of all forms of \ninfrastructure -Physical, Digital and Social\nDigital Public Infrastructure (DPI) -Promoted \nformalisation and financial inclusion\nDeepening and widening of tax base via GST\nStrengthened financial sector brought savings, credit \nand Investment back on track\nGIFT  IFSC - A robust  gateway  for global  capital  and \nfinancial  services  for the economy\nProactive  Inflation  management\nAll parts  of country  becoming  active  participants  in \neconomic  growth\n-10-505101520\nFY14

In [13]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [14]:
len(texts)

20

In [15]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

<ipython-input-15-f838166cf7bd>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [16]:
document_search = FAISS.from_texts(texts, embeddings)

In [17]:
document_search

In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [19]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

<ipython-input-19-25e042d5cc81>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  chain = load_qa_chain(OpenAI(), chain_type="stuff")
<ipython-input-19-25e042d5cc81>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: 

In [21]:
query = "Vision for Amrit Kaal"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nThe vision for Amrit Kaal is to achieve inclusive development through strategies such as promoting private and public investment in post-harvest activities, expanding the application of Nano-DAP in all agro-climatic zones, and formulating the Atmanirbhar Oilseeds Abhiyaan to achieve self-sufficiency in oilseeds. Additionally, the government aims to improve physical infrastructure through initiatives such as doubling FDI inflow and increasing capital expenditure, as well as promoting urban transformation via Metro rail and the NaMo Bharat program. Other key features include the promotion of people-centric inclusive development, strengthening of the financial sector, and the development of all forms of infrastructure (physical, digital, and social) in order to achieve a prosperous Bharat in harmony with nature.'

In [22]:
query = "How much the agriculture target will be increased to and what the focus will be"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The agriculture target will be increased to 2023-24 and the focus will be on promoting private and public investment in post-harvest activities, expanding the application of Nano-DAP in all agro-climatic zones, and formulating the Atmanirbhar Oilseeds Abhiyaan strategy to achieve self-sufficiency in oilseeds.'